In [8]:
suppressPackageStartupMessages({
    library(Seurat)
    library(celldex)
    library(dplyr)
    library(SingleR)
    library(openxlsx)
    library(dplyr)
})

In [4]:
load(file=file.path("objects", "02.rda"))

In [5]:
# Function to clean and truncate sheet names if they are too long
clean_sheetname <- function(name, max_length = 31) {
    if (nchar(name) > max_length) {
        name <- trimws(name)
        name <- gsub("[^A-Za-z0-9]", "", name)
    }
    return(name)
}

In [6]:
# Set the identifier for cell types
obj <- SetIdent(obj, value = "fine.type")

# Define the broad immune cell types you are interested in
immune <- c('T Cell', 'NK Cell', 'Plasma Cell', 'B Cell', 'Myeloid Cell')

# Loop over each broad immune cell type
for (broad_type in immune) {
    # Create a new workbook for each broad type
    
    wb <- createWorkbook()
    
    # Subset the object for the current broad type
    obj_subset <- subset(obj, broad.type == broad_type)
    
    # Get unique fine types for the current broad type
    fine_types <- unique(obj_subset@meta.data$fine.type)
    
    # Loop over each fine type
    for (fine_type in fine_types) {
        # Prepare and possibly clean the sheet name
        sheetname <- gsub("^\\s*\\d+\\.\\s*", "", fine_type)
        sheetname <- clean_sheetname(sheetname)

        if (fine_type == "7. IGHG+ IGL- Plasma") sheetname <- paste("7. IGHG+ IGL- Plasma", "2")
        
        # Add a new sheet for each fine type within the broad type
        addWorksheet(wb, sheetName = sheetname)

        # Run FindMarkers for the current fine type
        cell_markers <- FindMarkers(obj_subset, ident.1 = fine_type) %>% filter(0.01 >= p_val_adj)

        # Write the data to the corresponding sheet
        writeData(wb, sheet = sheetname, x = cell_markers, rowNames = TRUE)
    }
    
    # Save the workbook to a file named after the broad type
    filename <- paste("AMP_II_SLE", gsub(" ", "_", broad_type), "Markers.xlsx", sep = "_")
    saveWorkbook(wb, file = file.path("Markers", "Immune", filename), overwrite = TRUE)
}


In [16]:
# Set the identifier for cell types
obj <- SetIdent(obj, value = "fine.type")

# Define the broad structural cell types you are interested in
structural <- setdiff(unique(obj@meta.data$broad.type), immune)
structural <- structural[-3] # GLOM is only one cell type

for (broad_type in structural) {
    # Create a new workbook for each broad type
    
    wb <- createWorkbook()
    
    # Subset the object for the current broad type
    obj_subset <- subset(obj, broad.type == broad_type)
    
    # Get unique fine types for the current broad type
    fine_types <- unique(obj_subset@meta.data$fine.type)
    
    # Loop over each fine type
    for (fine_type in fine_types) {
        # Prepare and possibly clean the sheet name
        sheetname <- gsub("^\\s*\\d+\\.\\s*", "", fine_type)
        sheetname <- clean_sheetname(sheetname)

        # Add a new sheet for each fine type within the broad type
        addWorksheet(wb, sheetName = sheetname)

        # Run FindMarkers for the current fine type
        cell_markers <- FindMarkers(obj_subset, ident.1 = fine_type) %>% filter(0.01 >= p_val_adj)

        # Write the data to the corresponding sheet
        writeData(wb, sheet = sheetname, x = cell_markers, rowNames = TRUE)
    }
    
    # Save the workbook to a file named after the broad type
    filename <- paste("AMP_II_SLE", gsub(" ", "_", broad_type), "Markers.xlsx", sep = "_")
    saveWorkbook(wb, file = file.path("Markers", "Structural", filename), overwrite = TRUE)
}

In [6]:
immune <- c('T Cell', 'NK Cell', 'Plasma Cell', 'B Cell', 'Myeloid Cell')

In [9]:
structural <- setdiff(unique(obj@meta.data$broad.type), immune)
structural_cells <- subset(obj, broad.type %in% structural)
structural_cells <- SetIdent(structural_cells, value = "broad.type")
cell_markers <- FindMarkers(structural_cells, ident.1 = "GLOM") %>% filter(0.01 >= p_val_adj)

wb <- createWorkbook()
addWorksheet(wb, sheetName = 'GLOM')
writeData(wb, sheet = 'GLOM', x = cell_markers, rowNames = TRUE)
filename <- "AMP_II_SLE_GLOM_Markers.xlsx"
saveWorkbook(wb, file = file.path("Markers", "Structural", filename), overwrite = TRUE)